### Import all the required libraries

In [1]:
import pandas as pd 
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

### get the data from file to dataframe

In [2]:
df = pd.read_csv("concrete_data.csv")
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
df.shape

(1030, 9)

In [4]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


### To clean and validate the data check for null and datatype of the columns

In [5]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
df.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash               float64
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                     int64
Strength              float64
dtype: object

### Split the data to predictors(input) and target(output) Columns

In [7]:
#Regression model
predictors = df[df.columns[df.columns != 'Strength']]
n_cols = predictors.shape[1]
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target = df['Strength']
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Declare our regression Model

In [9]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')  # Set the mean_square_error for validating the dataset
    return model

### Evaluate our Model

In [10]:
predictions = []

# Run the loop 50 times
for n_try in range(50): 
    #split data with 30% test set
    X_train, X_test, y_train, y_test = train_test_split(predictors,target,test_size=0.3,random_state=n_try) # random state in re_initialize for each try

    # build the model
    model = regression_model()

    # fit the model with 50 epochs 
    model.fit(X_train,y_train,epochs=50,verbose=0)

    # evaluate the model
    pred_loss_mse = model.evaluate(X_test, y_test, verbose=0)  # the loss will be the mean_squeared_error as we have set our compilation to mse
    y_pred = model.predict(X_test)
    mean_squared_error = metrics.mean_squared_error(y_test, y_pred) # Calculate MSE with sklearn
    predictions.append([pred_loss_mse,mean_squared_error])

pred_df = pd.DataFrame(predictions, columns = ['Model_Eval_MSE','Model_Pred_MSE'])  
pred_df.head()

,Model_Eval_MSE,Model_Pred_MSE
0,506.749786,506.749783
1,108.024147,108.024150
2,310.402161,310.402152
3,249.771317,249.771291
4,164.791794,164.791818


In [11]:
np.mean(pred_df['Model_Pred_MSE'])

408.5124535165206

In [12]:
np.std(pred_df['Model_Pred_MSE'])

605.792354576414